# 경사 하강법을 이용한 얕은 신경망 만들기

In [1]:
import tensorflow as tf
import numpy as np

## 하이퍼 파라미터 설정

In [2]:
EPOCHS = 1000

## 네트워크 구조 정의
### 얕은 신경망
#### 입력 계층 : 2, 은닉 계층 : 128 (Sigmoid activation), 출력 계층 : 10 (Softmax activation)

1. tf.keras.Model 상속 -> 자신만의 모델 생성
2. super(MyModel,self) -> 파이썬에는 다중 상속을 허용하기 때문에 한번의 부모 생성자를 실행하기 위해
3. tf.keras.layers.Dense -> fully connected layer('층갯수',activate function)를 만들어 놨음

In [3]:
class MyModel(tf.keras.Model):
    #생성자 - 은닉 계층 설정
    def __init__(self):
        super(MyModel,self).__init__()
        self.d1 = tf.keras.layers.Dense(128,activation='sigmoid')
        self.d2 = tf.keras.layers.Dense(10,activation='softmax')
    # 데이터 처리 및 학습
    def __call__(self,x,Training = None,Mask = None):
        x = self.d1(x) # 은닉계층에 x 넣음
        return self.d2(x) # 활성계층에서 출력값 나온다.
    

## 학습 루프 정의
tf.function 데코레이터<br>
@tf.function을 함수에 붙여줄 경우, 여전히 다른 일반 함수들처럼 사용할 수 있습니다. 하지만 그래프 내에서 컴파일 되었을 때는 더 빠르게 실행하고, GPU나 TPU를 사용해서 작동하고, 세이브드모델(SavedModel)로 내보내는 것이 가능해집니다.

#### 매개 변수 설명
loss_object : loss fun<br>
train_metric : 목표를 얼마나 달성했는지<br><br>
#### 중간 함수 설명
tf.GradientTape() : <br>
 텐서플로는 자동 미분(주어진 입력 변수에 대한 연산의 그래디언트(gradient)를 계산하는 것)을 위한 tf.GradientTape API를 제공합니다. tf.GradientTape는 컨텍스트(context) 안에서 실행된 모든 연산을 테이프(tape)에 "기록"합니다. 그 다음 텐서플로는 후진 방식 자동 미분
(reverse mode differentiation)을 사용해 테이프에 "기록된" 연산의 그래디언트를 계산합니다.

In [4]:
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_metric):
    with tf.GradientTape() as tape:
        predictions = model(inputs)
        loss = loss_object(labels,predictions)
    gradients = tape.gradient(loss,model.trainable_variables) # d(loss)/dx -> scalar / vector -> vector
    
    optimizer.apply_gradients(zip(gradients,model.trainable_variables)) #각 학습 변수와 각 변수 대한 loss 변화율을 optimizer에 적용
    train_loss(loss)
    train_metric(labels,predictions)

## 데이터셋 생성, 전처리

#### 파이썬 *
가변 인자를 사용할 때 * 연산을 사용한다. <br>
def hi(*number):<br>
이때, number에 packing을하고 진행한다.<br>
반대로 hi(*list)는 list에있는 것을 unpacking하고 진행한다.<br>
*center_pt.shape -> [1,2]를 1,2 매개변수를 넣는 것과 동일

In [5]:
np.random.seed(0)

pts = list()
labels = list()
center_pts = np.random.uniform(-8.0, 8.0, (10, 2)) # -8,8사이 값을 가지는 10,2 생성
for label, center_pt in enumerate(center_pts):
    for _ in range(100):
        pts.append(center_pt + np.random.randn(*center_pt.shape))
        labels.append(label)

pts = np.stack(pts, axis=0).astype(np.float32)
labels = np.stack(labels, axis=0)

train_ds = tf.data.Dataset.from_tensor_slices((pts, labels)).shuffle(1000).batch(32)

In [6]:
pts.shape

(1000, 2)

## 모델 생성

In [7]:
model = MyModel()

## 손실 함수 및 최적화 알고리즘 설정
### CrossEntropy, Adam Optimizer
Sparse -> 몇 번 째가 1인지<br>
Adam -> gradient를 이용한 optimizer 중 하나 (각 변수의 학습률을 고려하는 gradient 방식)

In [8]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

## 평가 지표 설정
### Accuracy


In [9]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

## 데이터셋 및 학습 파라미터 저장

In [10]:
for epoch in range(EPOCHS):
    for x,label in train_ds:
        train_step(model,x,label,loss_object,optimizer,train_loss,train_accuracy)
        
        template = 'EPOCHS : {}, Loss : {}, Accuracy : {}'.format(epoch+1,train_loss.result(),train_accuracy.result()*100)
    print(template)
        

EPOCHS : 1, Loss : 2.156022548675537, Accuracy : 24.5
EPOCHS : 2, Loss : 1.9644743204116821, Accuracy : 37.70000076293945
EPOCHS : 3, Loss : 1.82964026927948, Accuracy : 43.33333206176758
EPOCHS : 4, Loss : 1.7226595878601074, Accuracy : 49.42499923706055
EPOCHS : 5, Loss : 1.6316007375717163, Accuracy : 53.320003509521484
EPOCHS : 6, Loss : 1.5566028356552124, Accuracy : 56.36666488647461
EPOCHS : 7, Loss : 1.4912461042404175, Accuracy : 59.55714416503906
EPOCHS : 8, Loss : 1.4336810111999512, Accuracy : 62.212501525878906
EPOCHS : 9, Loss : 1.3812156915664673, Accuracy : 64.14444732666016
EPOCHS : 10, Loss : 1.3338840007781982, Accuracy : 65.95999908447266
EPOCHS : 11, Loss : 1.2912176847457886, Accuracy : 67.41818237304688
EPOCHS : 12, Loss : 1.25199294090271, Accuracy : 68.875
EPOCHS : 13, Loss : 1.216082215309143, Accuracy : 70.05384826660156
EPOCHS : 14, Loss : 1.1830689907073975, Accuracy : 71.27143096923828
EPOCHS : 15, Loss : 1.1520075798034668, Accuracy : 72.1866683959961
EPO

EPOCHS : 122, Loss : 0.45922207832336426, Accuracy : 86.32623291015625
EPOCHS : 123, Loss : 0.4578675925731659, Accuracy : 86.34959411621094
EPOCHS : 124, Loss : 0.4564898610115051, Accuracy : 86.37822723388672
EPOCHS : 125, Loss : 0.4550530016422272, Accuracy : 86.4000015258789
EPOCHS : 126, Loss : 0.4537101984024048, Accuracy : 86.42301940917969
EPOCHS : 127, Loss : 0.45239391922950745, Accuracy : 86.44252014160156
EPOCHS : 128, Loss : 0.45107778906822205, Accuracy : 86.4632797241211
EPOCHS : 129, Loss : 0.44977423548698425, Accuracy : 86.48372650146484
EPOCHS : 130, Loss : 0.4484322965145111, Accuracy : 86.50614929199219
EPOCHS : 131, Loss : 0.44712167978286743, Accuracy : 86.52671813964844
EPOCHS : 132, Loss : 0.4459013342857361, Accuracy : 86.54696655273438
EPOCHS : 133, Loss : 0.4445987343788147, Accuracy : 86.56541442871094
EPOCHS : 134, Loss : 0.4433186948299408, Accuracy : 86.58507537841797
EPOCHS : 135, Loss : 0.44207534193992615, Accuracy : 86.60369873046875
EPOCHS : 136, Lo

EPOCHS : 240, Loss : 0.365866094827652, Accuracy : 87.711669921875
EPOCHS : 241, Loss : 0.3654361665248871, Accuracy : 87.7170181274414
EPOCHS : 242, Loss : 0.36501047015190125, Accuracy : 87.72148132324219
EPOCHS : 243, Loss : 0.36457788944244385, Accuracy : 87.72715759277344
EPOCHS : 244, Loss : 0.3641393184661865, Accuracy : 87.73401641845703
EPOCHS : 245, Loss : 0.3637257218360901, Accuracy : 87.73918151855469
EPOCHS : 246, Loss : 0.363310307264328, Accuracy : 87.74674224853516
EPOCHS : 247, Loss : 0.36288073658943176, Accuracy : 87.75465393066406
EPOCHS : 248, Loss : 0.3624853491783142, Accuracy : 87.75968170166016
EPOCHS : 249, Loss : 0.36208024621009827, Accuracy : 87.76506042480469
EPOCHS : 250, Loss : 0.36166268587112427, Accuracy : 87.76959991455078
EPOCHS : 251, Loss : 0.3612797260284424, Accuracy : 87.77569580078125
EPOCHS : 252, Loss : 0.36089348793029785, Accuracy : 87.78055572509766
EPOCHS : 253, Loss : 0.3605078458786011, Accuracy : 87.78656005859375
EPOCHS : 254, Loss 

EPOCHS : 358, Loss : 0.33114585280418396, Accuracy : 88.21815490722656
EPOCHS : 359, Loss : 0.3309286832809448, Accuracy : 88.21922302246094
EPOCHS : 360, Loss : 0.33073297142982483, Accuracy : 88.2236099243164
EPOCHS : 361, Loss : 0.33053532242774963, Accuracy : 88.2271499633789
EPOCHS : 362, Loss : 0.3303333818912506, Accuracy : 88.23121643066406
EPOCHS : 363, Loss : 0.3301357626914978, Accuracy : 88.23443603515625
EPOCHS : 364, Loss : 0.3299247920513153, Accuracy : 88.23764038085938
EPOCHS : 365, Loss : 0.3297332227230072, Accuracy : 88.24027252197266
EPOCHS : 366, Loss : 0.32951897382736206, Accuracy : 88.24262237548828
EPOCHS : 367, Loss : 0.329325795173645, Accuracy : 88.24578094482422
EPOCHS : 368, Loss : 0.32913342118263245, Accuracy : 88.24755859375
EPOCHS : 369, Loss : 0.32893943786621094, Accuracy : 88.25257110595703
EPOCHS : 370, Loss : 0.328731894493103, Accuracy : 88.25513458251953
EPOCHS : 371, Loss : 0.3285287916660309, Accuracy : 88.25875854492188
EPOCHS : 372, Loss : 

EPOCHS : 475, Loss : 0.31280842423439026, Accuracy : 88.50147247314453
EPOCHS : 476, Loss : 0.31269150972366333, Accuracy : 88.50357055664062
EPOCHS : 477, Loss : 0.31256383657455444, Accuracy : 88.50566101074219
EPOCHS : 478, Loss : 0.3124474883079529, Accuracy : 88.50711822509766
EPOCHS : 479, Loss : 0.3123542070388794, Accuracy : 88.50834655761719
EPOCHS : 480, Loss : 0.3122304677963257, Accuracy : 88.51062774658203
EPOCHS : 481, Loss : 0.3121066689491272, Accuracy : 88.51123046875
EPOCHS : 482, Loss : 0.3119819164276123, Accuracy : 88.5145263671875
EPOCHS : 483, Loss : 0.31185826659202576, Accuracy : 88.51676940917969
EPOCHS : 484, Loss : 0.311745822429657, Accuracy : 88.51900482177734
EPOCHS : 485, Loss : 0.3116157352924347, Accuracy : 88.52103424072266
EPOCHS : 486, Loss : 0.31149664521217346, Accuracy : 88.52387237548828
EPOCHS : 487, Loss : 0.311396986246109, Accuracy : 88.52587127685547
EPOCHS : 488, Loss : 0.3112730383872986, Accuracy : 88.52766418457031
EPOCHS : 489, Loss : 

EPOCHS : 593, Loss : 0.3013285994529724, Accuracy : 88.70033264160156
EPOCHS : 594, Loss : 0.30124935507774353, Accuracy : 88.70235443115234
EPOCHS : 595, Loss : 0.30118030309677124, Accuracy : 88.70336151123047
EPOCHS : 596, Loss : 0.30110791325569153, Accuracy : 88.70486450195312
EPOCHS : 597, Loss : 0.30103832483291626, Accuracy : 88.70569610595703
EPOCHS : 598, Loss : 0.30095651745796204, Accuracy : 88.70685577392578
EPOCHS : 599, Loss : 0.30087903141975403, Accuracy : 88.70834350585938
EPOCHS : 600, Loss : 0.30079385638237, Accuracy : 88.71033477783203
EPOCHS : 601, Loss : 0.30073320865631104, Accuracy : 88.71198272705078
EPOCHS : 602, Loss : 0.3006582260131836, Accuracy : 88.71328735351562
EPOCHS : 603, Loss : 0.30057492852211, Accuracy : 88.71525573730469
EPOCHS : 604, Loss : 0.3005000352859497, Accuracy : 88.71721649169922
EPOCHS : 605, Loss : 0.30044952034950256, Accuracy : 88.71834564208984
EPOCHS : 606, Loss : 0.3003985285758972, Accuracy : 88.71980285644531
EPOCHS : 607, Lo

EPOCHS : 711, Loss : 0.29318949580192566, Accuracy : 88.85372924804688
EPOCHS : 712, Loss : 0.29312875866889954, Accuracy : 88.85505676269531
EPOCHS : 713, Loss : 0.2930682301521301, Accuracy : 88.85638427734375
EPOCHS : 714, Loss : 0.29301008582115173, Accuracy : 88.85798645019531
EPOCHS : 715, Loss : 0.29295381903648376, Accuracy : 88.85902404785156
EPOCHS : 716, Loss : 0.2928939759731293, Accuracy : 88.85977935791016
EPOCHS : 717, Loss : 0.29284197092056274, Accuracy : 88.8612289428711
EPOCHS : 718, Loss : 0.2927834391593933, Accuracy : 88.86211395263672
EPOCHS : 719, Loss : 0.2927212417125702, Accuracy : 88.86314392089844
EPOCHS : 720, Loss : 0.2926589548587799, Accuracy : 88.86430358886719
EPOCHS : 721, Loss : 0.2926110327243805, Accuracy : 88.8656005859375
EPOCHS : 722, Loss : 0.292549729347229, Accuracy : 88.86648559570312
EPOCHS : 723, Loss : 0.2924908399581909, Accuracy : 88.8677749633789
EPOCHS : 724, Loss : 0.29243016242980957, Accuracy : 88.86933898925781
EPOCHS : 725, Loss

EPOCHS : 829, Loss : 0.2871685028076172, Accuracy : 88.98528289794922
EPOCHS : 830, Loss : 0.2871167063713074, Accuracy : 88.98638153076172
EPOCHS : 831, Loss : 0.28706830739974976, Accuracy : 88.98724365234375
EPOCHS : 832, Loss : 0.28702816367149353, Accuracy : 88.98858642578125
EPOCHS : 833, Loss : 0.2869829535484314, Accuracy : 88.98931884765625
EPOCHS : 834, Loss : 0.2869426906108856, Accuracy : 88.99028778076172
EPOCHS : 835, Loss : 0.2868932783603668, Accuracy : 88.99125671386719
EPOCHS : 836, Loss : 0.2868458926677704, Accuracy : 88.99198913574219
EPOCHS : 837, Loss : 0.28679826855659485, Accuracy : 88.99354553222656
EPOCHS : 838, Loss : 0.28674617409706116, Accuracy : 88.99462890625
EPOCHS : 839, Loss : 0.2867043614387512, Accuracy : 88.9958267211914
EPOCHS : 840, Loss : 0.2866637408733368, Accuracy : 88.99702453613281
EPOCHS : 841, Loss : 0.286614328622818, Accuracy : 88.99749755859375
EPOCHS : 842, Loss : 0.2865690588951111, Accuracy : 88.99798583984375
EPOCHS : 843, Loss : 

EPOCHS : 947, Loss : 0.28240254521369934, Accuracy : 89.10337829589844
EPOCHS : 948, Loss : 0.2823620140552521, Accuracy : 89.1039047241211
EPOCHS : 949, Loss : 0.28232866525650024, Accuracy : 89.10526275634766
EPOCHS : 950, Loss : 0.2822907865047455, Accuracy : 89.10620880126953
EPOCHS : 951, Loss : 0.28224825859069824, Accuracy : 89.10725402832031
EPOCHS : 952, Loss : 0.28220662474632263, Accuracy : 89.10882568359375
EPOCHS : 953, Loss : 0.28216439485549927, Accuracy : 89.10997009277344
EPOCHS : 954, Loss : 0.28212231397628784, Accuracy : 89.11079406738281
EPOCHS : 955, Loss : 0.28208816051483154, Accuracy : 89.11183166503906
EPOCHS : 956, Loss : 0.28205448389053345, Accuracy : 89.11286926269531
EPOCHS : 957, Loss : 0.2820171117782593, Accuracy : 89.11410522460938
EPOCHS : 958, Loss : 0.28197887539863586, Accuracy : 89.115234375
EPOCHS : 959, Loss : 0.2819373607635498, Accuracy : 89.1157455444336
EPOCHS : 960, Loss : 0.28189942240715027, Accuracy : 89.11698150634766
EPOCHS : 961, Los

## 데이터셋 및 학습 파라미터 저장

In [16]:

W_h, b_h = model.d1.get_weights()
W_o, b_o = model.d2.get_weights()
W_h = np.transpose(W_h)
W_o = np.transpose(W_o)
